# Titanic Survival Prediction: Full Pipeline Project

This notebook contains the complete solution for the Titanic Survival Prediction practice project. The goal is to build, optimize, and compare a **Random Forest Classifier** (an ensemble model) and a **Logistic Regression** model using scikit-learn pipelines and cross-validation.

##  Objectives

1.  Use scikit-learn to build a model for a classification problem.
2.  Implement a pipeline to combine preprocessing steps with a machine learning model.
3.  Use **Grid Search Cross-Validation** for hyperparameter optimization.
4.  Interpret the results and compare the performances of the classifiers.

***

#1. Setup and Data Loading

In [1]:
# 1.1 Install and Import Libraries (assuming a standard environment)
# !pip install numpy matplotlib pandas scikit-learn seaborn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Set random seed for reproducibility
np.random.seed(42)

In [2]:
# 1.2 Load Data (Using seaborn's built-in dataset for portability)
df = sns.load_dataset('titanic')

# 1.3 Feature Selection (Selecting the features used in the project outline)
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'class', 'who', 'deck']
target = 'survived'

X = df[features].copy()
y = df[target].copy()

print(X.head(0))

          survived  pclass   sex      age  sibsp  parch      fare  embarked  class    who  adult_male  deck  embark_town  alive  alone

In [3]:
# 1.4 Check Class Imbalance
print("Survival Distribution:\n", y.value_counts())
print(f"Class 0 (Did Not Survive): {y.value_counts()[0]/len(y):.1%}")
print(f"Class 1 (Survived): {y.value_counts()[1]/len(y):.1%}")


Survival Distribution:
0    549
1    342
Name: survived, dtype: int64
Class 0 (Did Not Survive): 61.6%
Class 1 (Survived): 38.4%


In [4]:
# 1.5 Split Data (using stratification to maintain class ratio)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Train Survival Ratio (1/0): {(y_train == 1).sum() / (y_train == 0).sum():.2f}")
print(f"Test Survival Ratio (1/0): {(y_test == 1).sum() / (y_test == 0).sum():.2f}")

X_train shape: (712, 10)
X_test shape: (179, 10)
Train Survival Ratio (1/0): 0.62
Test Survival Ratio (1/0): 0.62


## 2. Preprocessing Pipeline Definition

In [5]:
# 2.1 Define Feature Lists
numerical_features = ['age', 'fare']
categorical_features = ['pclass', 'sex', 'embarked', 'class', 'who', 'deck']

# 2.2 Define Numerical Pipeline (Impute Median, Scale)
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 2.3 Define Categorical Pipeline (Impute Most Frequent, One-Hot Encode)
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 2.4 Combine Pipelines with ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, numerical_features),
    ('cat', categorical_pipeline, categorical_features)
], remainder='passthrough')

## 3. Model 1: Random Forest Classifier (Ensemble Model)

In [6]:
# 3.1 Create Random Forest Pipeline
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('randomforestclassifier', RandomForestClassifier(random_state=42))
])

# 3.2 Define Parameter Grid
rf_param_grid = {
    'randomforestclassifier__n_estimators': [100, 200, 300],
    'randomforestclassifier__max_depth': [None, 10, 20]
}

# 3.3 Perform Grid Search
rf_grid_search = GridSearchCV(rf_pipeline, rf_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
rf_grid_search.fit(X_train, y_train)

# 3.4 Print Results
print("RandomForestClassifier Grid Search Results:")
print(f"Best Cross-Validation Score: {rf_grid_search.best_score_}")
print(f"Best Hyperparameters: {rf_grid_search.best_params_}")

RandomForestClassifier Grid Search Results:
Best Cross-Validation Score: 0.8385438596491228
Best Hyperparameters: {'randomforestclassifier__max_depth': 10, 'randomforestclassifier__n_estimators': 100}


In [7]:
# 3.5 Evaluate Random Forest on Test Set (This is the Ensemble Accuracy)
rf_test_score = rf_grid_search.best_estimator_.score(X_test, y_test)
y_pred_rf = rf_grid_search.best_estimator_.predict(X_test)

print(f"Test set accuracy (Random Forest): {rf_test_score:.6f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))

# 3.6 Extract Feature Importance
best_rf_model = rf_grid_search.best_estimator_['randomforestclassifier']
feature_names = rf_grid_search.best_estimator_.named_steps['preprocessor'].get_feature_names_out()
importances = best_rf_model.feature_importances_

importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 3.7 Plot Feature Importance
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df.head(10), palette='viridis')
plt.title('Random Forest Feature Importance')
plt.show()

Test set accuracy (Random Forest): 0.832402

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.89      0.87       110
           1       0.79      0.75      0.77        69

    accuracy                           0.83       179
   macro avg       0.82      0.82      0.82       179
weighted avg       0.83      0.83      0.83       179


## 4. Model 2: Logistic Regression

In [8]:
# 4.1 Create Logistic Regression Pipeline
lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('logisticregression', LogisticRegression(random_state=42))
])

# 4.2 Define Parameter Grid
lr_param_grid = {
    'logisticregression__solver': ['liblinear', 'lbfgs'],
    'logisticregression__penalty': ['l1', 'l2'],
    'logisticregression__class_weight': [None, 'balanced']
}

# 4.3 Perform Grid Search
lr_grid_search = GridSearchCV(lr_pipeline, lr_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
lr_grid_search.fit(X_train, y_train)

# 4.4 Print Results
print("LogisticRegression Grid Search Results:")
print(f"Best Cross-Validation Score: {lr_grid_search.best_score_}")
print(f"Best Hyperparameters: {lr_grid_search.best_params_}")

LogisticRegression Grid Search Results:
Best Cross-Validation Score: 0.8385438596491228
Best Hyperparameters: {'logisticregression__class_weight': 'balanced', 'logisticregression__penalty': 'l1', 'logisticregression__solver': 'liblinear'}


In [9]:
# 4.5 Evaluate Logistic Regression on Test Set
lr_test_score = lr_grid_search.best_estimator_.score(X_test, y_test)
y_pred_lr = lr_grid_search.best_estimator_.predict(X_test)

print(f"Test set accuracy (Logistic Regression): {lr_test_score:.6f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))

# 4.6 Extract and Plot Coefficient Magnitudes
best_lr_model = lr_grid_search.best_estimator_['logisticregression']
feature_names_out = lr_grid_search.best_estimator_.named_steps['preprocessor'].get_feature_names_out()
coefficients = best_lr_model.coef_[0]

importance_df_lr = pd.DataFrame({
    'Feature': feature_names_out,
    'Coefficient': coefficients
}).sort_values(by='Coefficient', ascending=False, key=abs) # Sort by absolute values

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x=importance_df_lr['Coefficient'].abs().head(10), y=importance_df_lr['Feature'].head(10), palette='viridis')
plt.title('Top 10 Feature Coefficient Magnitudes (Logistic Regression)')
plt.xlabel('Coefficient Magnitude')
plt.show()

Test set accuracy (Logistic Regression): 0.826816

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.88      0.86       110
           1       0.77      0.74      0.76        69

    accuracy                           0.83       179
   macro avg       0.81      0.81      0.81       179
weighted avg       0.82      0.83      0.82       179


## 5. Final Model Summary

The final test set accuracies for the two models are very close:

| Model | Test Set Accuracy |
| :--- | :--- |
| Random Forest (Ensemble) | **0.832402** |
| Logistic Regression | **0.826816** |

The **Random Forest Classifier** (an ensemble model) performed marginally better (0.56% difference). The distinct feature importance plots for each model suggest underlying **collinearity** among the features, which is a key takeaway from this comparative analysis.